# HUẤN LUYỆN MÔ HÌNH SVM
Thực hiện huấn luyện mô hình SVM dựa trên các đặc trưng đã được trích suất từ ANN để phát hiện giao dịch gian lận.


#### Khai báo thư viện

In [ ]:
# Import các thư viện
import os
import pandas as pd
import numpy as np
from keras import Model
from keras.src.saving import load_model
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import joblib
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler

#### Khai báo đường dẫn

In [ ]:
# Đường dẫn dữ liệu và mô hình
split_dir = "../data/old_data/splits"
model_dir = "../models"
svm_model_path = f"{model_dir}/svm_model.pkl"
plot_dir = "../plots"

### Tải dữ liệu
Dữ liệu train và test được đọc từ các file đã chia.


In [ ]:
# Đọc dữ liệu
X_train = pd.read_csv(f"{split_dir}/X_train.csv")
y_train = pd.read_csv(f"{split_dir}/y_train.csv")
X_test = pd.read_csv(f"{split_dir}/X_test.csv")
y_test = pd.read_csv(f"{split_dir}/y_test.csv")

# Kiểm tra dữ liệu
print(f"Kích thước tập train: {X_train.shape}")
print(f"Kích thước tập test: {X_test.shape}")


### Tải và trích xuất đặc trưng từ ANN

In [ ]:
# Tải mô hình ANN đã huấn luyện
ann_model = load_model(f"{model_dir}/ann_model.keras")

# Xem kiến trúc mô hình đã tải
ann_model.summary()

In [ ]:
feature_extractor = Model(inputs=ann_model.inputs, outputs=ann_model.layers[-2].output)

# Trích xuất đặc trưng
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

print(f"Kích thước đặc trưng tập train: {X_train_features.shape}")
print(f"Kích thước đặc trưng tập test: {X_test_features.shape}")


### Huấn luyện SVM
Mô hình SVM được cấu hình với: Kernel: Linear.


In [ ]:
# Khởi tạo và huấn luyện SVM
svm_model = SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)
svm_model.fit(X_train_features, y_train.values.ravel())


### Lưu mô hình SVM
Mô hình được lưu vào file `models/svm_model.pkl`.


In [ ]:
# Lưu mô hình SVM
joblib.dump(svm_model, svm_model_path)
print(f"Mô hình SVM đã được lưu tại: {svm_model_path}")

### Đánh giá mô hình

#### Ma trận nhầm lẫn (Confusion Matrix)

In [ ]:
# Dự đoán trên tập test
y_test_pred = svm_model.predict(X_test_features)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:\n", conf_matrix)

# Báo cáo chi tiết
print("Classification Report:\n", classification_report(y_test, y_test_pred))

disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["Không Gian Lận", "Gian Lận"])
disp.plot(cmap='Blues', values_format='d')
plt.title("Ma Trận Nhầm Lẫn")
plt.savefig(os.path.join(f"{plot_dir}/evaluation", "confusion_matrix.png"))
plt.show()



#### Biểu đồ ROC

In [ ]:
svm_model = joblib.load(svm_model_path)

In [ ]:
# Tính ROC Curve
fpr, tpr, _ = roc_curve(y_test, svm_model.decision_function(X_test))
roc_auc = auc(fpr, tpr)

# Vẽ Biểu Đồ ROC
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.title("Biểu Đồ ROC")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.savefig(os.path.join(f"{plot_dir}/evaluation", "roc_curve.png"))
plt.show()